### Question 1: 
#### Implement the uncollapsed Gibbs sampler for latent Dirichlet allocation we discussed in class. Apply it to state-of-the-union addresses at a level of aggregation you choose and describe the output.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
import nltk
from nltk.corpus import stopwords
from nltk import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import lda
from collections import Counter
import scipy.sparse as ssp
import topicmodels as tpm
from nltk import word_tokenize
from numpy.random import dirichlet
import time


### Question 2: 
#### Find and run a version of the collapsed Gibbs sampler using lda in python. Run it for the same parameter values (i.e. Dirichlet hyperparameters and K) and documents you chose for the question above. 

In [ ]:
# Import the original data
data_origin = pd.read_table("~/Documents/text_mining_course/text_mining/PS1/speech_data_extend.txt", encoding="utf-8") 
dt_matrix = pd.DataFrame.from_csv("~/Documents/dt_matrix.csv", sep = ";", index_col = 0) 

X = dt_matrix.iloc[:][data_origin.year >= 1945]
X = ssp.csr_matrix(X.astype(int))

K = 2

Col_Gibbs = tpm.LDA.LDAGibbs(prep_data,K)

Col_Gibbs.alpha
Col_Gibbs.beta

burn_samples = 1000
jumps = 50
used_samples = 10

Col_Gibbs.sample(burn_samples,jumps,used_samples)



#### a) Plot the perplexity across sampling iterations beginning from several diﬀerent starting values for each algorithm. Which algorithm appears to burn in faster?

In [ ]:
Col_Gibbs.perplexity() 

word_topics = Col_Gibbs.tt 
doc_topics = Col_Gibbs.dt 

#### b) For the best-ﬁt chain from each algorithm after the burn-in period, construct estimates of the predictive distribution of θd for each document across a number of draws. Recall that the kth element of the predictive distribution is the number of times topic k is observed in document d plus α over Kα + Nd. Are the average values of these predictive distributions similar in the uncollapsed and collapsed samplers? How variable are these predictive distributions in the two algorithms across sample draws?

### Question 3: 
#### Compare the classiﬁcation performance of a penalized logistic regression (which is implemented in scikit learn) when paragraphs are represented as unigram counts over raw terms versus topic shares. Use training samples to estimate the relationship between document content and political party, and then assess its out-of-sample performance on held-out data. If you choose to tune the penalization parameter by cross validation, ensure that this is done only using training data.

In [ ]:
# Select only documents which appear after 1945
X = dt_matrix.iloc[:][data_origin.year >= 1945]
X = X.reset_index()
X = X.drop('level_0', axis=1)

# Create a new variable with only presidents and years
parties = data_origin.loc[:,['president', 'year']]
parties = parties.reset_index()

# Create a new variable with 1 when presidents are Democrats, 0 when they are Republicans
zero_len = pd.Series(np.zeros(len(parties.index)))
parties['parties'] = zero_len
parties['parties'] = (parties.president == "Truman") | (parties.president == "Kennedy") | (parties.president == "Johnson") | (parties.president == "Carter") | (parties.president == "Clinton") | (parties.president == "Obama")
parties.parties = list(map(lambda x: 1 if x else 0, parties.parties))
parties = parties.iloc[:][parties.year >= 1945]

# Set the parties variable as y
y = parties.parties

# Split the X and y variables into a training and test set
X_train, X_test, y_train, y_test = train_test_split(X.as_matrix(), y, test_size = 0.3, random_state = 42)

# Create the logistic regression estimator with an l1 loss parameter
log_reg = LogisticRegression(penalty = "l1")

# Set some parameters to tune over
c_space = [1.3, 1.5, 1.7]
parameters = {'C': c_space}

# Create a cross-validation estimator
cv = GridSearchCV(log_reg, parameters)

# Fit the model, predict and report the accuracy and the best parameter
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)
print("Accuracy: {}".format(cv.score(X_test, y_test)))
print(classification_report(y_test, y_pred))
print("Tuned Model Parameters: {}".format(cv.best_params_))




## Run a logistic regression using the topics instead of the document-term matrix
X = dt_matrix.iloc[:][data_origin.year >= 1945]
X = ssp.csr_matrix(X.astype(int))

K, S, alpha, eta = 20, 1000, 0.1, 0.01

Col_Gibbs = lda.LDA(n_topics=K, n_iter=S, alpha=alpha, eta=eta)

X = Col_Gibbs.fit_transform(X)

# Split the X and y variables into a training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 59)

# Create the logistic regression estimator
log_reg = LogisticRegression("l2")

# Set some parameters to tune over
c_space = [0.5, 2, 5]
parameters = {'C': c_space}

# Create a cross-validation estimator
cv = GridSearchCV(log_reg, parameters)

# Fit the model, predict and report the accuracy and the best parameter
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)
print("Accuracy: {}".format(cv.score(X_test, y_test)))
print(classification_report(y_test, y_pred))
print("Tuned Model Parameters: {}".format(cv.best_params_))